In [1]:
import re
import string
import numpy as np
import math
import nltk
import io

from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import islice
from rouge import Rouge



In [2]:
# nltk.download('stopwords')
# nltk.download()  # uncomment these lines once they are not downloaded

In [3]:
def preprocessing(article):
    
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    article_preprocessed = []
    sentences = sent_tokenize(article)
    for sentence in sentences:
        sentence_preprocessed = []
        sentence = re.sub(r"[^a-zA-Z\s]+", "", sentence)
        words = word_tokenize(sentence)
        for word in words:
            if (word not in stopwords_english and word not in string.punctuation):
                word_stemmed = stemmer.stem(word)  
                sentence_preprocessed.append(word_stemmed)
#         sentence_preprocessed = " ".join(sentence_preprocessed)
        article_preprocessed.append(sentence_preprocessed)
            
        
        
#     words = [word_tokenize(sent) for sent in sentences]
#     words_without_stopwords = [[word for word in sent if word not in stopwords.words('english')] for sent in words]
    
    return article_preprocessed


In [4]:
article_file = io.open("articles/original (" + str(1) +").txt")
article_file.readline()
article = article_file.read()
article_file.close()

# print(article)
article_preprocessed = preprocessing(article)
print(article_preprocessed)
print(len(article_preprocessed))

[['quarterli', 'profit', 'us', 'media', 'giant', 'timewarn', 'jump', 'bn', 'three', 'month', 'decemb', 'yearearli'], ['the', 'firm', 'one', 'biggest', 'investor', 'googl', 'benefit', 'sale', 'highspe', 'internet', 'connect', 'higher', 'advert', 'sale'], ['timewarn', 'said', 'fourth', 'quarter', 'sale', 'rose', 'bn', 'bn'], ['it', 'profit', 'buoy', 'oneoff', 'gain', 'offset', 'profit', 'dip', 'warner', 'bro', 'less', 'user', 'aol'], ['time', 'warner', 'said', 'friday', 'own', 'searchengin', 'googl'], ['but', 'internet', 'busi', 'aol', 'mix', 'fortun'], ['it', 'lost', 'subscrib', 'fourth', 'quarter', 'profit', 'lower', 'preced', 'three', 'quarter'], ['howev', 'compani', 'said', 'aol', 'underli', 'profit', 'except', 'item', 'rose', 'back', 'stronger', 'internet', 'advertis', 'revenu'], ['it', 'hope', 'increas', 'subscrib', 'offer', 'onlin', 'servic', 'free', 'timewarn', 'internet', 'custom', 'tri', 'sign', 'aol', 'exist', 'custom', 'highspe', 'broadband'], ['timewarn', 'also', 'restat', '

In [5]:
def convert_list_to_string(sentences):  # converts list of lists to list of strings
    sentences_modified = []   # list of strings
    for sentence in sentences:
        sentence_modified = ''.join(sentence)
        sentences_modified.append(sentence_modified)
    return sentences_modified

In [6]:
### Feature 1

def calculate_TF_IDF(content):
    flat_words = [word for sent in content for word in sent]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(flat_words)
    feature_names = tfidf_vectorizer.get_feature_names()
    word_scores = {}
    for i, feature in enumerate(feature_names):
        word_scores[feature] = tfidf_matrix[:, i].sum()
    return word_scores
    

In [7]:
def calculate_each_sentence_score(article_preprocessed, word_scores):
    sentence_scores = []
    for sent in article_preprocessed:
        score = 0
        for word in sent:
            score += word_scores.get(word, 0)
        sentence_scores.append(score)
        
    sentence_scores = sentence_scores / max(sentence_scores)
    return sentence_scores

In [8]:
def generate_summary(sentences, sentence_scores, num_sentences):
    top_sentences_idx = sorted(range(len(sentence_scores)), key=lambda i: sentence_scores[i], reverse=True)[:num_sentences]
    summary = [sentences[i] for i in top_sentences_idx]
    return ' '.join(summary)

In [9]:
def test_tf_idf(file_name): # using tf-idf
    
    article_file = io.open("articles/original (" + str(file_name) +").txt")
    article_file.readline()
    article = article_file.read()
    article_file.close()
    
    sentences = sent_tokenize(article)
    sentences[0] = sentences[0][1:]
    
    summarized_file = io.open("articles/summarized (" + str(file_name) +").txt")
    summarized_original = summarized_file.read()
    summarized_file.close()

    article_preprocessed = preprocessing(article)
    word_scores = calculate_TF_IDF(article_preprocessed)
    sentence_scores = calculate_each_sentence_score(article_preprocessed, word_scores)
    summary = generate_summary(sentences, sentence_scores, 7)
    
    print(sentence_scores)
    rouge = Rouge(metrics=['rouge-n', 'rouge-l'], max_n=2)
    scores_tf_idf = rouge.get_scores(summary, summarized_original)
    
    rouge_1_tf_idf = scores_tf_idf['rouge-1']['f']
    rouge_2_tf_idf = scores_tf_idf['rouge-2']['f']
    rouge_l_tf_idf = scores_tf_idf['rouge-l']['f']
    print('t-idf accuracy')
    print('Rouge 1 score is: %f' % (rouge_1_tf_idf))
    print('Rouge 2 score is: %f' % (rouge_2_tf_idf))
    print('Rouge l score is: %f' % (rouge_l_tf_idf))

In [10]:
test_tf_idf(301)

[0.75       0.28571429 0.64285714 0.96428571 0.85714286 0.60714286
 0.75       0.82142857 0.64285714 1.         0.64285714 0.32142857
 0.35714286 0.96428571]
t-idf accuracy
Rouge 1 score is: 0.709402
Rouge 2 score is: 0.629310
Rouge l score is: 0.529915


C:\Users\moham\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
### Feature 2

def sentence_length(article_preprocessed):
    article_preprocessed = convert_list_to_string(article_preprocessed)
    max_length = 0
    for sentence in article_preprocessed:
        # print(sentence)
        if len(sentence) > max_length:
            max_length = len(sentence)
            
    sentence_length_feature = []
    for sentence in article_preprocessed:
        sentence_length_feature.append(len(sentence) / max_length)
    return sentence_length_feature

In [12]:
article_file = io.open("articles/original (" + str(1) +").txt")
article_file.readline()
art = article_file.read()
article_file.close()

art = preprocessing(art)
print(len(art))
art = convert_list_to_string(art)
sentence_length(article_preprocessed)

20


[0.559322033898305,
 0.6694915254237288,
 0.3135593220338983,
 0.4830508474576271,
 0.3305084745762712,
 0.2288135593220339,
 0.4745762711864407,
 0.6779661016949152,
 0.847457627118644,
 0.6016949152542372,
 0.4745762711864407,
 1.0,
 0.423728813559322,
 0.8728813559322034,
 0.652542372881356,
 0.5084745762711864,
 0.3389830508474576,
 0.5338983050847458,
 0.8220338983050848,
 0.3050847457627119]

In [13]:
def generate_X_labels(article_preprocessed):
    # feature 1 (tf_idf)
    word_scores = calculate_TF_IDF(article_preprocessed)
    tf_idf_score_feature = calculate_each_sentence_score(article_preprocessed, word_scores)
    
    # feature 2 (sentence_length)
    sentence_length_feature = sentence_length(article_preprocessed)
    
    matrix = np.column_stack((tf_idf_score_feature, sentence_length_feature))

    return matrix

In [14]:
article_file = io.open("articles/original (" + str(1) +").txt")
article_file.readline()
article = article_file.read()
article_file.close()

art = preprocessing(article)
art = convert_list_to_string(art)
print(generate_X_labels(article_preprocessed))
print(len(generate_X_labels(article_preprocessed)))

[[0.79069767 0.55932203]
 [0.62790698 0.66949153]
 [0.81395349 0.31355932]
 [0.97674419 0.48305085]
 [0.34883721 0.33050847]
 [0.37209302 0.22881356]
 [0.76744186 0.47457627]
 [0.95348837 0.6779661 ]
 [1.         0.84745763]
 [0.58139535 0.60169492]
 [0.60465116 0.47457627]
 [0.81395349 1.        ]
 [0.88372093 0.42372881]
 [0.53488372 0.87288136]
 [0.81395349 0.65254237]
 [0.62790698 0.50847458]
 [0.39534884 0.33898305]
 [0.48837209 0.53389831]
 [0.86046512 0.8220339 ]
 [0.65116279 0.30508475]]
20


In [15]:
def generate_Y_labels(original, summarized):
    Y_list = []
    original_sentences = sent_tokenize(original)
    original_sentences[0] = original_sentences[0][1:] # to remove the \n
    summarized_sentences = sent_tokenize(summarized)
    
    for original_sentence in original_sentences:
        added = 0
        for summarized_sentence in summarized_sentences:
            if original_sentence in summarized_sentence:
                Y_list.append(1)
                added = 1
                break
        if added == 0:
            Y_list.append(0)
    
    return Y_list, original_sentences
    

In [16]:
article_file = io.open("articles/original (" + str(301) +").txt")
article_file.readline()
article = article_file.read()
article_file.close()

summarized_file = io.open("articles/summarized (" + str(301) +").txt")
summarized = summarized_file.read()
summarized_file.close()

Y,_ = generate_Y_labels(article, summarized)
print(Y)
print(len(Y))

[1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0]
14


In [17]:
X_matrix = []
X = []
Y = []
sentences = []

for i in range (1, 101):   # loading business articles
    article_file = io.open("train_original/business/article (" + str(i) +").txt")
    article_file.readline()
    article = article_file.read()
    article_file.close()

    summarized_file = io.open("train_summary/business/summary (" + str(i) +").txt")
    summarized = summarized_file.read()
    summarized_file.close()
    
    article_preprocessed = preprocessing(article)
#     article_preprocessed = convert_list_to_string(article_preprocessed)
    X_i = generate_X_labels(article_preprocessed)
    Y_i, original_list_no_first_space = generate_Y_labels(article, summarized)
    
    if(len(X_i) != len(Y_i)):
        print('Error! features and labels are not equal in length')
        
    Y.extend(Y_i)
    X_matrix.extend(X_i)
    sentences.extend(original_list_no_first_space)
    
for i in range (1, 101):   # loading entertainment articles
    article_file = io.open("train_original/entertainment/article (" + str(i) +").txt")
    article_file.readline()
    article = article_file.read()
    article_file.close()

    summarized_file = io.open("train_summary/entertainment/summary (" + str(i) +").txt")
    summarized = summarized_file.read()
    summarized_file.close()
    
    article_preprocessed = preprocessing(article)
#     article_preprocessed = convert_list_to_string(article_preprocessed)
    X_i = generate_X_labels(article_preprocessed)
    Y_i, original_list_no_first_space = generate_Y_labels(article, summarized)
    
    if(len(X_i) != len(Y_i)):
        print('Error! features and labels are not equal in length')
        
    Y.extend(Y_i)
    X_matrix.extend(X_i)
    sentences.extend(original_list_no_first_space)
    
for i in range (1, 101):   # loading politics articles
    article_file = io.open("train_original/politics/article (" + str(i) +").txt")
    article_file.readline()
    article = article_file.read()
    article_file.close()

    summarized_file = io.open("train_summary/politics/summary (" + str(i) +").txt")
    summarized = summarized_file.read()
    summarized_file.close()
    
    article_preprocessed = preprocessing(article)
#     article_preprocessed = convert_list_to_string(article_preprocessed)
    X_i = generate_X_labels(article_preprocessed)
    Y_i, original_list_no_first_space = generate_Y_labels(article, summarized)
    
    if(len(X_i) != len(Y_i)):
        print('Error! features and labels are not equal in length')
        
    Y.extend(Y_i)
    X_matrix.extend(X_i)
    sentences.extend(original_list_no_first_space)
    
for i in range (1, 101):   # loading sport articles
    article_file = io.open("train_original/sport/article (" + str(i) +").txt")
    article_file.readline()
    article = article_file.read()
    article_file.close()

    summarized_file = io.open("train_summary/sport/summary (" + str(i) +").txt")
    summarized = summarized_file.read()
    summarized_file.close()
    
    article_preprocessed = preprocessing(article)
#     article_preprocessed = convert_list_to_string(article_preprocessed)
    X_i = generate_X_labels(article_preprocessed)
    Y_i, original_list_no_first_space = generate_Y_labels(article, summarized)
    
    if(len(X_i) != len(Y_i)):
        print('Error! features and labels are not equal in length')
        
    Y.extend(Y_i)
    X_matrix.extend(X_i)
    sentences.extend(original_list_no_first_space)
    
for i in range (1, 101):   # loading tech articles
    article_file = io.open("train_original/tech/article (" + str(i) +").txt")
    article_file.readline()
    article = article_file.read()
    article_file.close()

    summarized_file = io.open("train_summary/tech/summary (" + str(i) +").txt")
    summarized = summarized_file.read()
    summarized_file.close()
    
    article_preprocessed = preprocessing(article)
#     article_preprocessed = convert_list_to_string(article_preprocessed)
    X_i = generate_X_labels(article_preprocessed)
    Y_i, original_list_no_first_space = generate_Y_labels(article, summarized)
    
    if(len(X_i) != len(Y_i)):
        print('Error! features and labels are not equal in length')
        
    Y.extend(Y_i)
    X_matrix.extend(X_i)
    sentences.extend(original_list_no_first_space)

for x in X_matrix:
    X.append(x.tolist())
    
X = np.matrix(X)

m = len(X)
# m = 500
print(len(X))
print(len(Y))

8576
8576


In [18]:
nn_input_dim = 2 # input layer size (we have two input features)
nn_output_dim = 1  # output layer size (we have one output)

# Gradient descent parameters
alpha = 0.1  # learning rate for gradient descent
# print(Y)
# print(X)

In [19]:
def sigmoid(x):
    # TODO 1: Compute the sigmoid function at the given x (~1 line)
    # For example: sigmoid(2) should compute the value of sigmoid function at x = 2.
    # Hint: Use np.exp instead of math.exp to allow for vectorization.
    #----------------------------------------------------------------------------------------------
    sig = (1/(1+np.exp(-x)))
    #----------------------------------------------------------------------------------------------
    
    return sig


In [20]:
def build_model(nn_hdim, num_passes=20000, print_loss=False):
    
    np.random.seed(0)
    W1 = np.random.randn(nn_hdim, nn_input_dim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((nn_hdim, 1))
    W2 = np.random.randn(nn_output_dim, nn_hdim) / np.sqrt(nn_hdim)
    b2 = np.zeros((nn_output_dim, 1))

    model = {}

    for i in range(0, num_passes):
        DW1 = 0
        DW2 = 0
        Db1 = 0
        Db2 = 0
        cost = 0

        for j in range(0, m):
            a0 = X[j, :].reshape(-1, 1)  # Every training example is a column vector.
            y = Y[j]
            
            z1 = np.dot(W1 , a0 )+ b1
            a1 = np.tanh(z1)
            z2 = np.dot(W2 , a1) + b2
            a2 = sigmoid(z2)
            
#             if (i == num_passes -1 ):
#                 print('True value: %f, got: %f'% (y, a2))

            cost_j = -1 * ((np.log(a2) * y + (1-y)* np.log(1-a2)))

            da2 =  ( -y/a2  + (1-y)/(1-a2) )
            dz2 =  da2 * a2 * ( 1 - a2)
            dW2 = np.dot(dz2 , a1.T)
            db2 = dz2

            da1 =  np.dot(dz2,W2).T
            dz1 = np.multiply(da1 , 1 - np.square(a1) )
            dW1 = np.dot(dz1 , a0.T )
            db1 = dz1

            DW1 += dW1
            DW2 += dW2
            Db2 += db2
            Db1 += db1
            cost += cost_j
        
        DW1 /= m
        DW2 /= m
        Db1 /= m
        Db2 /= m
        cost /= m

        W1 -= alpha * DW1
        b1 -= alpha * Db1
        W2 -= alpha * DW2
        b2 -= alpha * Db2

        model = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}

        if print_loss and i % 1000 == 0:
            print("Loss after iteration %i: %f" % (i, cost))

    return model

In [21]:
# Helper function to predict an output (0 or 1)
def predict(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    a0 = x.T
    
    # TODO 6 (aka TODO 2): Apply forward propagation on every test example a0 (a column vector 2x1) with its
    #  corresponding label y. It is required to compute z1, a1, z2, and a2  (SAME AS TODO2).
    # -----------------------------------------------------------------------------------------------
    z1 = np.dot(W1 , a0) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(W2 , a1) + b2
    a2 = sigmoid(z2)
    # ------------------------------------------------------------------------------------------------
    # Applying a threshold of 0.5 (i.e. predictions greater than 0.5 are mapped to 1, and 0 otherwise)
#     prediction = np.round(a2)
    prediction = a2
    
    return prediction

In [22]:
model = build_model(nn_hdim=5, num_passes=10001, print_loss=True)

Loss after iteration 0: 0.784817
Loss after iteration 1000: 0.496629
Loss after iteration 2000: 0.494060
Loss after iteration 3000: 0.493428
Loss after iteration 4000: 0.492979
Loss after iteration 5000: 0.492672
Loss after iteration 6000: 0.492431
Loss after iteration 7000: 0.492218
Loss after iteration 8000: 0.492019
Loss after iteration 9000: 0.491833
Loss after iteration 10000: 0.491665


In [32]:
def test(article_preprocessed_test, original_test, summarized_text, compression_ratio, file_number):
#     article_file = io.open("articles/original (" + str(file_number) +").txt")
#     article_file.readline()
#     original_test = article_file.read()
#     article_preprocessed_test = preprocessing(original_test)
#     article_file.close()

#     summarized_file = io.open("articles/summarized (" + str(file_number) +").txt")
#     summarized_text = summarized_file.read()
#     summarized_file.close()
    
    X_test = generate_X_labels(article_preprocessed_test)
    predicton = predict(model, X_test)
    Y_test, original_sentences = generate_Y_labels(original_test, summarized_text)
    num_sentences_summarized = math.ceil(compression_ratio * len(original_sentences))
    
    
    highest = np.argsort(predicton[0]) [::-1]
    highest = highest[: num_sentences_summarized]
#     highest = sorted(highest) # uncomment to arrange the article
    output_sentences = []
    output_indices = []
    

    
    for i in range (0, num_sentences_summarized):
        output_sentences.append(original_sentences[highest[i]])
        output_indices.append(highest[i])
        
    output_sentences = ''.join(output_sentences)
    
    rouge = Rouge(metrics=['rouge-n', 'rouge-l'], max_n=2)
    scores_nn = rouge.get_scores(output_sentences, summarized_text)
    
    rouge_1_nn = scores_nn['rouge-1']['f']
    rouge_2_nn = scores_nn['rouge-2']['f']
    rouge_l_nn = scores_nn['rouge-l']['f']
    
#     print('article number: %d' % (file_number))
#     print('nn accuracy')
#     print('Rouge 1 score is: %f' % (rouge_1_nn))
#     print('Rouge 2 score is: %f' % (rouge_2_nn))
#     print('Rouge l score is: %f' % (rouge_l_nn))
    
    
    
    ### classified using TF_idf score

    output_tf_idf = extractive_summary(original_test, num_sentences_summarized)
    
    scores_tf_idf = rouge.get_scores(output_tf_idf, summarized_text)
    
    rouge_1_tf_idf = scores_tf_idf['rouge-1']['f']
    rouge_2_tf_idf = scores_tf_idf['rouge-2']['f']
    rouge_l_tf_idf = scores_tf_idf['rouge-l']['f']
    
#     print('t-idf accuracy')
#     print('Rouge 1 score is: %f' % (rouge_1_tf_idf))
#     print('Rouge 2 score is: %f' % (rouge_2_tf_idf))
#     print('Rouge l score is: %f' % (rouge_l_tf_idf))
    
    return rouge_1_nn, rouge_2_nn, rouge_l_nn, rouge_1_tf_idf, rouge_2_tf_idf, rouge_l_tf_idf

In [33]:
def extractive_summary(text, num_sentences):
    # Preprocess the text
    text = text.lower()
    text = ''.join(c for c in text if c not in '1234567890')
    sentences = sent_tokenize(text)
    words = [word_tokenize(sent) for sent in sentences]
    words_without_stopwords = [[word for word in sent if word not in stopwords.words('english')] for sent in words]
    
    # Calculate TF-IDF scores
    flat_words = [word for sent in words_without_stopwords for word in sent]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(flat_words)
    feature_names = tfidf_vectorizer.get_feature_names()
    word_scores = {}
    for i, feature in enumerate(feature_names):
        word_scores[feature] = tfidf_matrix[:, i].sum()
    
    # Calculate sentence scores
    sentence_scores = []
    for sent in words_without_stopwords:
        score = 0
        for word in sent:
            score += word_scores.get(word, 0)
        sentence_scores.append(score)
    
    # Select top N sentences with highest scores
    top_sentences_idx = sorted(range(len(sentence_scores)), key=lambda i: sentence_scores[i], reverse=True)[:num_sentences]
    summary = [sentences[i] for i in top_sentences_idx]
    return ' '.join(summary)


In [35]:
# precision_nn = []
# recall_nn = []
# precision_tf_idf = []
# recall_tf_idf = []

rouge_1_list_nn = []
rouge_2_list_nn = []
rouge_l_list_nn = []

rouge_1_list_tf_idf = []
rouge_2_list_tf_idf = []
rouge_l_list_tf_idf = []

article_types = ['business', 'entertainment', 'politics', 'sport', 'tech']

for article_type in article_types:
    for i in range(101, 131):

        article_file = io.open("train_original/" + article_type + "/article (" + str(i) +").txt")
        article_file.readline()
        article = article_file.read()
        article_preprocessed = preprocessing(article)
        article_file.close()

        summarized_file = io.open("train_summary/" + article_type + "/summary (" + str(i) +").txt")
        summarized = summarized_file.read()
        summarized_file.close()

        rouge_1_nn, rouge_2_nn, rouge_l_nn, rouge_1_tf_idf, rouge_2_tf_idf, rouge_l_tf_idf = test(
            article_preprocessed, article, summarized,0.35, i)

        rouge_1_list_nn.append(rouge_1_nn)
        rouge_2_list_nn.append(rouge_2_nn)
        rouge_l_list_nn.append(rouge_l_nn)

        rouge_1_list_tf_idf.append(rouge_1_tf_idf)
        rouge_2_list_tf_idf.append(rouge_2_tf_idf)
        rouge_l_list_tf_idf.append(rouge_l_tf_idf)


print('Using nn')
print('Average Rouge 1 score is: %f' % (np.average(rouge_1_list_nn)))
print('Average Rouge 2 score is: %f' % (np.average(rouge_2_list_nn)))
print('Average Rouge l score is: %f' % (np.average(rouge_l_list_nn)))

print('Using tf_idf only')
print('Average Rouge 1 score is: %f' % (np.average(rouge_1_list_tf_idf)))
print('Average Rouge 2 score is: %f' % (np.average(rouge_2_list_tf_idf)))
print('Average Rouge l score is: %f' % (np.average(rouge_l_list_tf_idf)))

# print('Neural network accuracy: ')
# print('Average precision score is: %f' % (np.average(precision_nn)))
# print('Average recall score is: %f' % (np.average(recall_nn)))
      
# print('Classical approach accuracy using tf-idf: ')
# print('Average precision score is: %f' % (np.average(precision_tf_idf)))
# print('Average recall score is: %f' % (np.average(recall_tf_idf)))

Using nn
Average Rouge 1 score is: 0.768017
Average Rouge 2 score is: 0.693994
Average Rouge l score is: 0.561756
Using tf_idf only
Average Rouge 1 score is: 0.686983
Average Rouge 2 score is: 0.575671
Average Rouge l score is: 0.503824
